In [1]:
!pip install gym
!pip install stable-baselines3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 11.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [2]:
import torch
import torchvision # multiple dataset, model architecture, image transformation for computer vision
import torchvision.transforms as transforms

In [3]:
import numpy as np
import matplotlib.pyplot as plt
torch.torch.set_printoptions(linewidth=120)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
df = pd.read_csv('drive/My Drive/Notebooks/Pytorch/malware_detection_RL/dataset/data.csv')
df.head()

,Source,S_Port,Destination,D_Port,Packets,Bytes,Packets_sent,Bytes_sent,Packets_received,Bytes_received,Rel Start,Duration,Bits_sent_rate,Bits_received_rate,Label
0,172.30.6.255,57041,192.168.240.1,7680,3,198,3,198,0,0,1596.993566,9.007770,175.848184,0.000000,Benign
1,172.30.6.255,55638,35.186.249.84,80,86,67650,34,3483,52,64167,36.448377,22.163850,1257.182304,23160.958050,Benign
2,172.30.15.2348,51682,192.168.121.37,443,95,82309,34,3046,61,79263,5313.787170,4.092096,5954.894509,154958.241400,Benign
3,10.0.2.15,50719,119.254.87.201,80,11,1582,6,705,5,877,25327.813510,0.896225,6293.062568,7828.391308,Malicious
4,172.30.15.1555,50864,54.169.137.62,80,10,1888,6,998,4,890,3555.231347,1.830215,4362.329016,3890.253331,Benign


In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'])
# Resampling the minority class. The strategy can be changed as required.
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy='minority', random_state=42)
# Fit the model to generate the data.

feature_cols = ['Packets', 'Bytes','Packets_sent','Bytes_sent','Packets_received','Bytes_received','Bits_sent_rate','Bits_received_rate']
X = df[feature_cols] #Features
y = df['Label'] #target

oversampled_X, oversampled_Y = sm.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(oversampled_X, oversampled_Y, test_size = 0.3, random_state = 1)
#oversampled = pd.concat([pd.DataFrame(oversampled_Y), pd.DataFrame(oversampled_X)], axis=1)
#oversampled


In [7]:
from sklearn.preprocessing import normalize
X_train = normalize(X_train, norm = 'l1')
X_test = normalize(X_test, norm = 'l1')

column_names = ['Packets', 'Bytes','Packets_sent','Bytes_sent','Packets_received','Bytes_received','Bits_sent_rate','Bits_received_rate']

X_train = pd.DataFrame(X_train, columns = column_names)
X_test = pd.DataFrame(X_test, columns = column_names)
#y_trainn = pd.DataFrame(y_train)
#y_testt = pd.DataFrame(y_test)


In [ ]:
X_train.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Packets,Bytes,Packets_sent,Bytes_sent,Packets_received,Bytes_received,Bits_sent_rate,Bits_received_rate
0,0.001003,0.144554,0.000547,0.064297,0.000456,0.080257,0.315167,0.393719
1,0.001110,0.267173,0.000666,0.160171,0.000444,0.107003,0.277829,0.185604
2,0.000166,0.141233,0.000066,0.004477,0.000100,0.136757,0.022733,0.694467
3,0.001134,0.156006,0.000619,0.072693,0.000516,0.083313,0.319519,0.366200
4,0.001846,0.253959,0.001007,0.118335,0.000839,0.135624,0.227570,0.260818


In [8]:
import gymnasium
import numpy as np
from gymnasium import spaces


class CustomEnv(gymnasium.Env):
    """Custom Environment that follows gym interface."""

    metadata = {"render.modes": ["human"]}

    def __init__(self, data, label,history_t = 8):
      super(CustomEnv,self).__init__()
      # Define action and observation space
      # They must be gym.spaces objects
      # Example when using discrete actions:
      self.action_space = spaces.Discrete(2)
      # Example for using image as input (channel-first; channel-last also works):
      self.observation_space = spaces.Box(low=0, high=1,
                                          shape=(8,), dtype=np.float64)
      self.label = label.tolist()
      self.data = data
      self.history_t = history_t
      #self.reset()

    def step(self, action):

      feedback = abs(action - self.label[self.t]) #if label is 1 and action choose 1, reward becomes 1-1=0
      #the logic behind the reward function is that if the action and actual label are the same,
      #the reward would be always zero, returning a reward of +1 as defined in the if_else below

      self.reward=1
      if feedback != 0:
        self.reward = -1


      self.t += 1

      self.observation = []
      self.observation.append(self.data.iloc[self.t, : ]['Packets'])
      self.observation.append(self.data.iloc[self.t, : ]['Bytes'])
      self.observation.append(self.data.iloc[self.t, : ]['Packets_sent'])
      self.observation.append(self.data.iloc[self.t, : ]['Bytes_sent'])
      self.observation.append(self.data.iloc[self.t, : ]['Packets_received'])
      self.observation.append(self.data.iloc[self.t, : ]['Bytes_received'])
      self.observation.append(self.data.iloc[self.t, : ]['Bits_sent_rate'])
      self.observation.append(self.data.iloc[self.t, : ]['Bits_received_rate'])

      self.observation=np.array(self.observation)



      if (self.t==len(self.data)-1):
        self.done = True
      infos={}
      truncations=False
      #observations, self.rewards, terminations,truncations,infos;obs, reward, terminated, truncated, info
      return self.observation, self.reward, self.done,truncations,infos

    def reset(self, seed=None, options=None):
      self.t = 0
      self.done = False
      self.profits = 0
      self.position = []
      self.observation = []
      self.observation = [0.0 for _ in range(self.history_t)]
      self.observation=np.array(self.observation)

      infos={}

      return self.observation, infos  # reward, done, info can't be included


In [9]:
from stable_baselines3.common.env_checker import check_env

env = CustomEnv(X_train,y_train)
eval_env=CustomEnv(X_test,y_test)
# It will check your custom environment and output additional warnings if needed
check_env(env)
check_env(eval_env)

In [ ]:
'''

from sklearn import metrics

test_env = CustomEnv(X_test,y_test)
episodic_reward=[]
reward_LR=[]
accuracy_LR=[]
loss_LR=[]
def episodic_performance(model,episode):

  all_episode_rewards = []
  #env=model.get_env()

  obss = test_env.reset()
  episode_rewards = []
  done = False
  #obs = env.reset()
  y_pred=[]
  while not done:
      # _states are only useful when using LSTM policies
      action, _states = model.predict(obss)
      y_pred.append(action)

      # here, action, rewards and dones are arrays
      # because we are using vectorized env
      obss, reward, done, info = test_env.step(action)
      episode_rewards.append(reward)

  all_episode_rewards.append(sum(episode_rewards))

  mean_episode_reward = np.mean(all_episode_rewards)
  reward_LR.append(mean_episode_reward/3826)

  score=metrics.accuracy_score(y_test[:3826],y_pred)
  accuracy_LR.append(score)

  loss= metrics.log_loss(y_test[:3826],y_pred)
  loss_LR.append(loss)

  print("At Episode:",episode, 'Agent Mean Reward:',mean_episode_reward)
  print('Reward',reward_LR)
  print('Accuracy',accuracy_LR)
  print('loss',loss_LR) '''

'\n\nfrom sklearn import metrics\n\ntest_env = CustomEnv(X_test,y_test)\nepisodic_reward=[]\nreward_LR=[]\naccuracy_LR=[]\nloss_LR=[]\ndef episodic_performance(model,episode):\n  \n  all_episode_rewards = []\n  #env=model.get_env()\n \n  obss = test_env.reset()\n  episode_rewards = []\n  done = False\n  #obs = env.reset()\n  y_pred=[]\n  while not done:\n      # _states are only useful when using LSTM policies\n      action, _states = model.predict(obss)\n      y_pred.append(action)\n\n      # here, action, rewards and dones are arrays\n      # because we are using vectorized env\n      obss, reward, done, info = test_env.step(action)\n      episode_rewards.append(reward)\n\n  all_episode_rewards.append(sum(episode_rewards))\n\n  mean_episode_reward = np.mean(all_episode_rewards)\n  reward_LR.append(mean_episode_reward/3826)\n\n  score=metrics.accuracy_score(y_test[:3826],y_pred)\n  accuracy_LR.append(score)\n\n  loss= metrics.log_loss(y_test[:3826],y_pred)\n  loss_LR.append(loss)\n\n 

In [113]:
y_true = y_test.tolist()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [127]:
from stable_baselines3.common.callbacks import BaseCallback

from stable_baselines3.common.evaluation import evaluate_policy
from sklearn import metrics
import time
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score, precision_recall_curve,roc_auc_score

accuracy=[]
precision =[]
recall =[]
f1 = []
loss=[]
precision_recall=[]
roc_auc=[]
singleAction=[]
all_predicted_actions =[]
all_agent_rewards=[]
episode_counter = []
steps_counter =[]
training_time=[]



class CustomEvaluationCallback(BaseCallback):
    """
    A custom callback that derives from ``BaseCallback``.

    :param verbose: Verbosity level: 0 for no output, 1 for info messages, 2 for debug messages
    """
    def __init__(self,model,eval_env, verbose=1):
        super().__init__(verbose)
        self.env = eval_env
        self.model = model
        self.step_tracker =0
        self.episode = 0
        self.steps=0
        self.total_training_time=0
        self.start =0
        self.end=0

        # Those variables will be accessible in the callback
        # (they are defined in the base class)
        # The RL model
        # self.model = None  # type: BaseAlgorithm
        # An alias for self.model.get_env(), the environment used for training
        # self.training_env # type: VecEnv
        # Number of time the callback was called
        # self.n_calls = 0  # type: int
        # num_timesteps = n_envs * n times env.step() was called
        # self.num_timesteps = 0  # type: int
        # local and global variables
        # self.locals = {}  # type: Dict[str, Any]
        # self.globals = {}  # type: Dict[str, Any]
        # The logger object, used to report things in the terminal
        # self.logger # type: stable_baselines3.common.logger.Logger
        # Sometimes, for event callback, it is useful
        # to have access to the parent object
        # self.parent = None  # type: Optional[BaseCallback]

    def _on_training_start(self) -> None:
        """
        This method is called before the first rollout starts.

        """


        self.start= time.time()


    def _on_rollout_start(self) -> None:
        """
        A rollout is the collection of environment interaction
        using the current policy.
        This event is triggered before collecting new samples.
        """
        pass

    def _on_step(self) -> bool:

        """
        This method will be called by the model after each call to `env.step()`.

        For child callback (of an `EventCallback`), this will be called
        when the event is triggered.

        :return: If the callback returns False, training is aborted early.
        """
        #episodic_reward, episode_lengthe = evaluate_policy(self.model, self.env, n_eval_episodes=100, return_episode_rewards=True)

        self.step_tracker+=1
        #print(self.steps)
        self.steps+=1

        if (self.step_tracker ==500):
          all_predicted_actions.clear()
          self.step_tracker=0
          obss,_= self.env.reset(seed=1)
            #self.episode+=1

          done = False
          rewards=0

          while not done:
              #print(obss)
              act,_ = self.model.predict(obss)
              all_predicted_actions.append(act)

              obss, reward, done,_,_=self.env.step(act)

              rewards+=reward

          self.env.close()
          accuracy.append(metrics.accuracy_score(y_true[:6009],all_predicted_actions))
          loss.append(metrics.log_loss(y_true[:6009],all_predicted_actions))
          precision.append(metrics.precision_score(y_true[:6009],all_predicted_actions))
          f1.append(metrics.f1_score(y_true[:6009],all_predicted_actions))
          recall.append(metrics.recall_score(y_true[:6009],all_predicted_actions))
          precision_recall.append(metrics.precision_recall_curve(y_true[:6009],all_predicted_actions))
          roc_auc.append(metrics.roc_auc_score(y_true[:6009],all_predicted_actions))
          #all_agent_rewards.append(rewards)
        #steps_counter.append(self.steps)
        #episode_counter.append(self.episode)
          print("After ",self.steps,'steps, the accuracy is',accuracy)

        return True

    def _on_rollout_end(self) -> None:
        """
        This event is triggered before updating the policy.
        """
        pass

    def _on_training_end(self) -> None:
        """
        This event is triggered before exiting the `learn()` method.
        """
        self.end = time.time()
        self.total_training_time=self.end - self.start
        training_time.append(self.total_training_time)


In [118]:
from stable_baselines3.dqn.policies import DQNPolicy


class CustomDQNPolicy(DQNPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomDQNPolicy, self).__init__(*args, **kwargs,
                                          net_arch=[64,64],
                                          )
#100 50 25

In [128]:

from stable_baselines3 import PPO,A2C,DQN
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.logger import configure
from stable_baselines3.dqn.policies import DQNPolicy

def train(env, steps: int = 1, seed: int | None = 0, **env_kwargs):
    # Train a single model to play as each agent in a cooperative Parallel environment

    env.reset(seed=seed)

    print(f"Starting training")

    #eval_callback = EvalCallback(evalEnvv, eval_freq=1000)


# set up logger

    #default learning_rate=1e-3
    model =DQN(
      'MlpPolicy',
        env,
        verbose=1,
        learning_rate=1e-3,

    )

    custom_callback = CustomEvaluationCallback(model,eval_env)



    model.learn(total_timesteps=steps, callback=custom_callback, reset_num_timesteps=False, log_interval=4)

    #model.save(f"{env.unwrapped.metadata.get('name')}_{time.strftime('%Y%m%d-%H%M%S')}")

    #print("Model has been saved.")
    print("Finished training")
    #print(f"Finished training on {str(env.unwrapped.metadata['name'])}.")

    env.close()

In [129]:
env_kwargs = {}
train(env, steps=20000,seed=1)

Starting training
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
After  500 steps, the accuracy is [0.5877849891828923]
After  1000 steps, the accuracy is [0.5877849891828923, 0.6816442003661175]
After  1500 steps, the accuracy is [0.5877849891828923, 0.6816442003661175, 0.7713429855217174]
After  2000 steps, the accuracy is [0.5877849891828923, 0.6816442003661175, 0.7713429855217174, 0.8999833582958895]
After  2500 steps, the accuracy is [0.5877849891828923, 0.6816442003661175, 0.7713429855217174, 0.8999833582958895, 0.9137959727076053]
After  3000 steps, the accuracy is [0.5877849891828923, 0.6816442003661175, 0.7713429855217174, 0.8999833582958895, 0.9137959727076053, 0.9187884839407555]
After  3500 steps, the accuracy is [0.5877849891828923, 0.6816442003661175, 0.7713429855217174, 0.8999833582958895, 0.9137959727076053, 0.9187884839407555, 0.9184556498585456]
After  4000 steps, the accuracy is [0.5877849891828923, 0.6816442003661175, 0

In [110]:
all_agent_rewards
agent_reward=[reward/(6009) for reward in all_agent_rewards]
agent_reward

[0.16092527874854384,
 0.33000499251123316,
 0.5689798635380263,
 0.8156099184556499,
 0.8415709768680313,
 0.8412381427858213,
 0.836245631552671,
 0.8515559993343318,
 0.8582126809785322,
 0.8608753536362124,
 0.8625395240472624,
 0.8628723581294725,
 0.8701947079380928,
 0.8798468963221834,
 0.8731902146779831,
 0.873523048760193,
 0.8818439008154435,
 0.8868364120485938,
 0.8851722416375437,
 0.8845065734731237,
 0.8795140622399734,
 0.8868364120485938,
 0.8805125644866034,
 0.889166250624064,
 0.889166250624064,
 0.8818439008154435,
 0.8801797304043935,
 0.8851722416375437,
 0.8885005824596439,
 0.8881677483774338,
 0.891496089199534,
 0.8871692461308038,
 0.8888334165418539,
 0.8998169412547845,
 0.8858379098019638,
 0.8875020802130138,
 0.8944915959394242,
 0.8931602596105841,
 0.8865035779663838,
 0.8871692461308038]

In [132]:
precision_recall

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(array([0.4872691 , 0.57366874, 1.        ]),
  array([1.        , 0.59972678, 0.        ]),
  array([0, 1])),
 (array([0.4872691 , 0.66386826, 1.        ]),
  array([1.        , 0.70218579, 0.        ]),
  array([0, 1])),
 (array([0.4872691 , 0.75194553, 1.        ]),
  array([1.       , 0.7920082, 0.       ]),
  array([0, 1])),
 (array([0.4872691 , 0.88590381, 1.        ]),
  array([1.        , 0.91222678, 0.        ]),
  array([0, 1])),
 (array([0.4872691 , 0.90654521, 1.        ]),
  array([1.        , 0.91769126, 0.        ]),
  array([0, 1])),
 (array([0.4872691 , 0.91496599, 1.        ]),
  array([1.        , 0.91871585, 0.        ]),
  array([0, 1])),
 (array([0.4872691, 0.9090604, 1.       ]),
  array([1.        , 0.92520492, 0.        ]),
  array([0, 1])),
 (array([0.4872691 , 0.90366216, 1.        ]),
  array([1.        , 0.93545082, 0.        ]),
  array([0, 1])),
 (array([0.4872691 , 0.92294694, 1.        ]),
  array([1.        , 0.93271858, 0.        ]),
  array([0, 1]))

In [ ]:
training_time

[12.476660251617432]

In [130]:
import csv





steps = [100*5*(step+1) for step in range(len(agent_reward))]



def create_csv(header):

  with open("drive/My Drive/Notebooks/Pytorch/Performance/DRL/sklearn_metrics.csv", 'w', encoding='UTF8',  newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

    for i in range(len(steps)):

      #rla=[predicted[i],true_label[i]]
      rla=[steps[i],accuracy[i],loss[i],precision[i],roc_auc[i],precision_recall[i],recall[i],f1[i]]
      # write the data
      writer.writerow(rla)

In [131]:
header=['Steps','Accuracy','Loss','Precision','ROC_AUC','Precision_Recall','Recall','F1']


new_header = ['Steps' , 'Rewards']

create_csv(header)